In [1]:
import pandas as pd
import numpy as np
import regex as re

import warnings
warnings.filterwarnings("ignore")

In [2]:
full_hand = pd.read_csv('h5_i2_2019-2022_hand_harvest.csv')
full_weather = pd.read_csv('h5-i2_2016-2021_daily-weather.csv')

hand_19_21 = full_hand[full_hand['Year'] <= 2021][10:].reset_index().drop(columns = 'index')
weather_over = full_weather[full_weather['Year'] >= 2019][595:].reset_index().drop(columns = 'index')

yrmmdy = []
short_field = []
weather_ind = []

for row in range(len(hand_19_21)):
    date = re.findall('[^\/]+', hand_19_21.iloc[row]['Date'])
    month = date[0] if len(date[0]) == 2 else '0' + date[0]
    day = date[1] if len(date[1]) == 2 else '0' + date[1]
    new_date = int(date[2] + month + day)
    yrmmdy.append(new_date)
    
    short = hand_19_21.iloc[row]['Site'][:2]
    short_field.append(short)
    
    weather_info = weather_over[(weather_over['Date '] == new_date) & (weather_over['FIeld'] == short)]
    if len(weather_info) != 0:
        weather_ind.append(weather_info.index[0])
    
hand_19_21.loc[:, 'Date'] = yrmmdy
hand_19_21.loc[:, 'Field'] = short_field
# hand_under is 2019 to 2021 with the date in yrmmdy format and the general field type as column 'Field'

to_append = weather_over.iloc[weather_ind].reset_index().drop(columns = 'index')

combined = pd.concat([hand_19_21, to_append.drop(columns = ['Date ', 'FIeld', 'Crop', 'DoY', 'Year'])], axis = 1)
combined.to_csv('harvest_weather.csv')

print(combined.columns)
combined.head()

Index(['Date', 'DoY', 'Year', 'Crop', 'Site', 'Treatment', 'Rep', 'Sample',
       'Site.Rep', 'Percent_H2O', 'AGB_g_m2', 'Grain_yield_g_m2',
       'Grain_yield_kg_ha', 'Stover_g_m2', 'Harvest_index', 'Field',
       'Location', 'Sol_Rad_MJ_m2_d', 'T_min_C', 'T_max_C', 'PCPN_mm_d',
       'RH_f', 'Wind_spd_m_s'],
      dtype='object')


,Date,DoY,Year,Crop,Site,Treatment,Rep,Sample,Site.Rep,Percent_H2O,...,Stover_g_m2,Harvest_index,Field,Location,Sol_Rad_MJ_m2_d,T_min_C,T_max_C,PCPN_mm_d,RH_f,Wind_spd_m_s
0,20190821,233,2019,SpringWheat,I201,ASP,1,1,ASP.1,22.0,...,552,37.9,I2,NP,24.043932,11.08,25.63,0.0,0.587751,1.598792
1,20190821,233,2019,SpringWheat,I202,ASP,2,1,ASP.2,18.2,...,487,35.3,I2,NP,24.043932,11.08,25.63,0.0,0.587751,1.598792
2,20190821,233,2019,SpringWheat,I203,ASP,3,1,ASP.3,19.9,...,651,34.5,I2,NP,24.043932,11.08,25.63,0.0,0.587751,1.598792
3,20190821,233,2019,SpringWheat,I204,ASP,4,1,ASP.4,19.3,...,564,35.9,I2,NP,24.043932,11.08,25.63,0.0,0.587751,1.598792
4,20190821,233,2019,SpringWheat,I205,ASP,5,1,ASP.5,24.4,...,438,36.6,I2,NP,24.043932,11.08,25.63,0.0,0.587751,1.598792


In [3]:
full_flux = pd.read_csv('Mass Flux values 2019_2022 updated.csv').iloc[269:].reset_index().drop(columns = 'index')

# indices 0 through 295 are for the Mandan site; 296 and on are for the Morton site

for row in range(len(full_flux)):
    full_flux.loc[row, 'ParameterA'] = full_flux.loc[row, 'ParameterA'][5:]
    full_flux.loc[row, 'ParameterB'] = full_flux.loc[row, 'ParameterB'][5:]
    paramC = full_flux.loc[row, 'ParameterC'] 
    if pd.isnull(paramC) == True:
        full_flux.loc[row, 'ParameterC'] = 0
    else:
        full_flux.loc[row, 'ParameterC'] = full_flux.loc[row, 'ParameterC'][5:]
            
full_flux.head()

,Site,Treatment,Location,Block,Year,Date,Days.After.Planting,total.flux,R^2,Rsqr_cleaned,ParameterA,ParameterB,ParameterC
0,Morton,ASP,A1,A,2019,613,28,9.202690,0.967755,NaN,29.153507908864025,3.0121077769489704,0
1,Morton,ASP,A1,A,2019,703,48,0.087665,0.016792,NaN,0.08358431000903724,0.050952906256122414,0.06486282522971061
2,Morton,ASP,A1,A,2019,1030,167,0.594851,0.999258,0.594851,0.26822005635438456,2.2019622751680714,-1.0703888676966595
3,Morton,ASP,A2,A,2019,613,28,11.225600,0.998136,11.225600,46.38059731020047,-5.55858159434648,3.177800149244445
4,Morton,ASP,A2,A,2019,703,48,0.114644,0.256768,NaN,0.08850167274659015,0.5031003717648979,-0.008691969534286017


In [4]:
by_min = pd.read_excel('avg wind speed nwern.xlsx', sheet_name = 'Morton')
daily = pd.read_excel('avg wind speed nwern.xlsx', sheet_name = 'MortonDailyAVG')

# i have chosen to not read in the mandan data since none of the data is from that site
# if you want to see if just change the sheet name to 'Mandan' or 'MandanDailyAVG'
# also these places are like 30 miles apart

wind_speed = by_min.merge(daily, on = 'date')
wind_speed.head()

,date,time,AvgWS6_10M_m/s,Average of AvgWS6_10M_m/s
0,2020-05-28,08:00:00,9.00,9.132355
1,2020-05-28,08:01:00,10.50,9.132355
2,2020-05-28,08:02:00,9.00,9.132355
3,2020-05-28,08:03:00,7.75,9.132355
4,2020-05-28,08:04:00,9.84,9.132355


In [5]:
agb = pd.read_csv('I2_CC_AGB_2020-2022.csv')[:-1]
agb['Field'] = ['I2'] * agb.shape[0]
agb.head()

,STUDY,SITE,SITEID,DATE,TRT,CC_AGB,Field
0,LTAR_CCE,NP,NP_I201,09_10_2020,ASP,50.9,I2
1,LTAR_CCE,NP,NP_I202,09_10_2020,ASP,NaN,I2
2,LTAR_CCE,NP,NP_I203,09_10_2020,ASP,106.4,I2
3,LTAR_CCE,NP,NP_I204,09_10_2020,ASP,34.9,I2
4,LTAR_CCE,NP,NP_I205,09_10_2020,ASP,140.4,I2
